In [ ]:
import pandas as pd
from gensim.models import word2vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Load Data

In [ ]:

folder_path = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/'
lankadeepa_data_all = folder_path + 'corpus/new/preprocess_from_isuru/lankadeepa_comments_with_article_2.csv'
gossip_lanka_data_all = folder_path + 'corpus/new/preprocess_from_unicode_values/gossip_lanka_all_comments.csv'


num_features = 350
context = 5
word2vec_model = folder_path + "word_embedding/word2vec/source2_data_from_gosspiLanka_and_lankadeepa/" + str(num_features) + "/word2vec_" \
                 + str(num_features) + "_" + str(context)


In [ ]:
lankadeepa_data = pd.read_csv(lankadeepa_data_all)
gossipLanka_data = pd.read_csv(gossip_lanka_data_all)

In [ ]:
# !zip /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/corpus/backup/word_embedding_resources.zip corpus/new/preprocess_from_isuru/lankadeepa_comments_with_article_2.csv corpus/new/preprocess_from_unicode_values/gossip_lanka_all_comments.csv

In [ ]:
lankadeepa_comments = []
lankadeepa_articles = []
gossipLanka_comments = gossipLanka_data['comment']

for index, row in lankadeepa_data.iterrows():
  article = row['article']
  if isinstance(article, str) :
    lankadeepa_articles.append(article.replace('\u200d','').replace('\u200b',''))
  comments = []
  for comment in row['comment'].split(';'):
    comments.append(comment.replace('\u200d',''))
  lankadeepa_comments.extend(comments)


In [ ]:
lankadeepa_comments = pd.Series(lankadeepa_comments)
lankadeepa_articles = pd.Series(lankadeepa_articles)
gossipLanka_comments = gossipLanka_data['comment']

all_data = pd.concat([lankadeepa_comments, lankadeepa_articles, gossipLanka_comments])

In [ ]:
comments = []
for comment in comments:
  comments += to_separate_sentences(comment)


# Generate Word2Vec

In [ ]:
def generate_word2vec_model(data):
    comments = []
    for comment in data:
        comments += to_separate_sentences(comment)

    print("# of comments taken for building the model: " + str(len(comments)))

    downsampling = 1e-3  # Downsample setting for frequent words
    min_word_count = 1  # Minimum word count - if not occurred this much remove
    num_workers = 4  # Number of threads to run in parallel

    model = word2vec.Word2Vec(comments, workers=num_workers, size=num_features, min_count=min_word_count,
                              window=context, sample=downsampling, sg=1)
    model.init_sims(replace=True)  # If you don't plan to train the model any further

    return model

In [ ]:
# split a comment into sentences of words
def to_separate_sentences(comment):
    sentences = []
    raw_sentences = str(comment).split(".")
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 2:
            sentences.append(raw_sentence.split())
    return sentences

In [ ]:
def check_model_qulity(model, word):
    for s in model.most_similar(word):
        print(s[0])
    # for s in model.wv.most_similar(positive=['ගෑණි', 'ඔබතුමන්ට'], negative=['මිනිහා']):
    #     print(s[0])

In [ ]:
model = generate_word2vec_model(all_data)

# LOad Word2Vec MOdel

In [ ]:
model.save(word2vec_model)
# model = word2vec.Word2Vec.load(word2vec_model)

In [ ]:

check_model_qulity(model, 'නැහැ')
check_model_qulity(model, 'හොඳයි')
check_model_qulity(model, 'ඔබට')